<a href="https://colab.research.google.com/github/ATrublie/Summarizing-paragraphs-of-a-given-document-keywords-extraction/blob/main/Audio%20to%20text/Audio_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SpeechRecognition

In [ ]:
!pip install pydub

In [ ]:
import speech_recognition as sr

In [ ]:
from google.colab import files

uploaded = files.upload()

for AUDIO_FILE in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=AUDIO_FILE, length=len(uploaded[AUDIO_FILE])))

Saving fruitnveg.mp3 to fruitnveg.mp3
User uploaded file "fruitnveg.mp3" with length 1525570 bytes


In [ ]:
from pydub import AudioSegment
sound = AudioSegment.from_mp3(AUDIO_FILE)
sound.export(AUDIO_FILE, format="wav")

<_io.BufferedRandom name='fruitnveg.mp3'>

In [ ]:
r = sr.Recognizer()
with sr.AudioFile(AUDIO_FILE) as source:
    audio = r.record(source)  # read the entire audio file

In [ ]:
!apt-get install -y swig libpulse-dev
!pip install pocketsphinx

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
libpulse-dev is already the newest version (1:11.1-1ubuntu7.11).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(AUDIO_FILE)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [ ]:
path = AUDIO_FILE
print("Full text:", get_large_audio_transcription(path))

audio-chunks/chunk1.wav : But this week's podcast in english. com beginners podcast. 
audio-chunks/chunk2.wav : We are talking about fruit and veg. 
audio-chunks/chunk3.wav : No not in the supermarket not in the market. 
audio-chunks/chunk4.wav : In our garden we have a large garden where we grow fruit and vegetables fruit and veg. 
audio-chunks/chunk5.wav : And we grow everything from seeds in the spring we have broad beans. 
audio-chunks/chunk6.wav : The beans are very long and very big. 
audio-chunks/chunk7.wav : Watching the spring richard. we have lots in the summer we have two small red ones and large yellow. 
audio-chunks/chunk8.wav : And we have corn. 
audio-chunks/chunk9.wav : Bright yellow corn. 
audio-chunks/chunk10.wav : Berry suites. 
audio-chunks/chunk11.wav : And my favorite. 
audio-chunks/chunk12.wav : We also have potatoes ob jeans there. 
audio-chunks/chunk13.wav : Dark purple color. 
audio-chunks/chunk14.wav : We have bright red raspberries my favorites. 
audio-chunk

In [ ]:
with open('out.txt', 'w') as f:
    print('', get_large_audio_transcription(path), file=f) 

audio-chunks/chunk1.wav : But this week's podcast in english. com beginners podcast. 
audio-chunks/chunk2.wav : We are talking about fruit and veg. 
audio-chunks/chunk3.wav : No not in the supermarket not in the market. 
audio-chunks/chunk4.wav : In our garden we have a large garden where we grow fruit and vegetables fruit and veg. 
audio-chunks/chunk5.wav : And we grow everything from seeds in the spring we have broad beans. 
audio-chunks/chunk6.wav : The beans are very long and very big. 
audio-chunks/chunk7.wav : Watching the spring richard. we have lots in the summer we have two small red ones and large yellow. 
audio-chunks/chunk8.wav : And we have corn. 
audio-chunks/chunk9.wav : Bright yellow corn. 
audio-chunks/chunk10.wav : Berry suites. 
audio-chunks/chunk11.wav : And my favorite. 
audio-chunks/chunk12.wav : We also have potatoes ob jeans there. 
audio-chunks/chunk13.wav : Dark purple color. 
audio-chunks/chunk14.wav : We have bright red raspberries my favorites. 
audio-chunk

In [ ]:
#!/usr/bin/env python
# coding: utf-8
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
 
def read_article(file_name):
    file = open('out.txt', "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    nltk.download("stopwords")
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize text
    print("Summarize Text: \n", ". ".join(summarize_text))

# let's begin
generate_summary( "msft.txt", 2)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
 But this week's podcast in english
com beginners podcast
We are talking about fruit and veg
No not in the supermarket not in the market
In our garden we have a large garden where we grow fruit and vegetables fruit and veg
And we grow everything from seeds in the spring we have broad beans
The beans are very long and very big
Watching the spring richard
we have lots in the summer we have two small red ones and large yellow
And we have corn
Bright yellow corn
Berry suites
And my favorite
We also have potatoes ob jeans there
Dark purple color
We have bright red raspberries my favorites
Red strawberries and melon
Green melon green melon just plums blue plums but green on the inside
And also yellow ones
We have sweet peppers green ones the other ones 101 lovely orange peaches very sweet
In the autumn we have cauliflower with whiteheads
And then broccoli with blue-green hea